# Imports

In [ ]:
######## Imports ########
import sys
import math
import numpy as np
import scipy.io
import os
#import matplotlib.pyplot as plt
#from mpl_toolkits import mplot3d
from matplotlib import cm
from scipy.optimize import minimize
#from scipy.optimize import NonlinearConstraint
from scipy.optimize import Bounds
# %matplotlib inline
# plt.rcParams.update({'font.size': 15})
# Code to minimize the free energy of a 2D and 3D droplet interacting with actin filament

# Struct class for variables

In [ ]:
class SystemParams():
    # Vasp vol fraction in droplet and bulk
    volfrac=[0,0];
    #Total volume
    Vtotal = 0.0;
    # Note: Volume and area need to be self-consistent
    # effective volume of VASP droplet (from crystal)
    veff = 0.0;
    # Droplet volume
    Vdrop = 0.0;
    Vdropfactor = 0.0; #Vfactor stores Vdrop*3/(4*pi)
    # Droplet area
    Varea = 0.0;
    surface_T = [0,0,0]
    # mixing enthalpy const.
    mix_enth_cost = 0.0;
    kB_T = 0.0;
    ## Filament parameters
    # Number of beads
    Nbeads = 0;
    # length of a cylinder (nm)
    lcyl = 0.0;
    # Radius of a cylinder (nm)
    rcyl = 0.0;
    # bending constant of bundle
    kbend = 0.0;
    kbendfil = 0.0;
    # Bathe et al Nonlinear
    nonlinearbool = False;
    # stretching constant
    kstr = 100.0;
    ellipse_com = [200,200,200]
    #ellipse values. Used when testing wetting of filament
    ellipse =[200,200,200];
    dimval = 3; #2D vs 3D
    noisefactor = 1;
    Nfil=1;
    #nonlinear coupling factor for bundle bending stiffness
    alpha = 0.0
    #Gittes JCB 1993
    kflex_rigidity_f = 7.29*1e-26*1e12*1e18;# pN.nm^2, filament flexural rigidity

# Functions

In [ ]:
def areEqual(a,b):
    ZERO_PREC = 1e-3
    return abs(a-b)<ZERO_PREC

#ignores ellipsoid coordinates
#gets a matrix of vectors from the opt_var variable. opt_var stores coordinates as a 1-d array
def getveccoord(opt_var, sysparams):
    dim = sysparams.dimval
    N = sysparams.Nbeads
    coord = np.zeros((N,dim))
    for idx in range(0,N):
        if(dim==3):
            coord[idx,:] = [opt_var[2+idx*dim], opt_var[2+idx*dim + 1], opt_var[2+idx*dim + 2]]
        else:
            coord[idx,:] = [opt_var[1+idx*dim], opt_var[1+idx*dim + 1]]
    return coord

# Bundle properties takes dimension (1, Sysparams.Nbeads \times Sysparams.dimval) to give variable of dimension (Sysparams.Nbeads, Sysparams.dimval)
def getbundleprops(coord, sysparams, normstatus=False, comstatus=False):
    N = sysparams.Nbeads
    #get precalculated vector
    vector = np.zeros((N-1,sysparams.dimval))
    if(normstatus):
        normvector =  np.zeros((N-1,sysparams.dimval))
    if(comstatus):
        comvector =  np.zeros((N-1,sysparams.dimval))
    cyllen = np.zeros(N-1)
    # Calculate length
    cprev = coord[0,:]
    for i in range(0,N-1):
        cnext = coord[i+1,:]
        vector[i,:] = cnext - cprev
        cyllen[i] = np.linalg.norm(vector[i,:])
        if(normstatus):
            normvector[i,:] = vector[i,:]/cyllen[i]
        if(comstatus):
            comvector[i,:] = 0.5*(cprev+cnext)
        cprev = cnext
    returnvec = [cyllen,vector]
    if(normstatus):
        returnvec.append(normvector)
    if(comstatus):
        returnvec.append(comvector)
    return returnvec

# Bundle and droplet property functions 

In [ ]:
def getsurfaceTension(e, sparam):
    tanharg = 64*(e - 1)
    # if tanharg <0, points are inside the ellipsoid
    # else points are outside the ellipsoid
    a = sparam.surface_T[1] #Actin-VASP
    b = sparam.surface_T[2] #Actin-PEG
    Delta = (b-a)/2
    return Delta*np.tanh(tanharg)+Delta+a

def getkbend(sysparams):
    if(sysparams.nonlinearbool==False):
        Nfil=sysparams.Nfil;
        kflex_bundle=sysparams.kflex_rigidity_f*Nfil;
        kbend_bundle = kflex_bundle/sysparams.lcyl
        return kbend_bundle
    #Assumptions
    df = 10;#filament cross-section diameter
    delta = 50;#spacing between crosslinker along the bundle, nm
    t = 15;# thickness of single crosslinker, nm
    #L = 2*pi*sparams.*;# Bundle length
    kx=2.5; #pN.nm #crosslinker stretching stiffness

    #Constants from literature
    Ef = 2.6e9*1e12/(1e18);# Filament young's modulus pN/nm^2

    #Derived constants
    If = sysparams.kflex_rigidity_f/Ef
    Af = np.pi*(df/2)*(df/2);#Filament area

    kstrf = Ef*Af/delta#pN/nm, filament stretching stiffness
    #Use alpha instead of precalculated value.
    alpha = sysparams.alpha
    #if(sysparams.alpha<0):
    #    alpha = kx*L*L/(Ef*Af*delta) # inter filament coupling paramter, dimensionless
    chisq = Af*(df+t)*(df+t)/(12*If)
    kBT =4.11 #pN.nm
    Nfil=sysparams.Nfil;
    kflex_bundle=sysparams.kflex_rigidity_f*Nfil;
    if(alpha>0):
        for j in range(1,11):
            cqj = j*j*np.pi*np.pi/12
            kflex_bundle = kflex_bundle + sysparams.kflex_rigidity_f*Nfil*(chisq*(Nfil-1)/(1+cqj*(Nfil+np.sqrt(Nfil))/alpha))
    kbend_bundle = kflex_bundle/sysparams.lcyl
    return kbend_bundle
######## Functions for Ellipse Geometry ########
def getellipsevolume(opt_var):
    return np.pi*opt_var[0]*opt_var[1];

def getellipsevolume3D(opt_var):
    return (4/3)*np.pi*opt_var[0]*opt_var[1]*opt_var[2];

# Gives the fraction of the line that |r1-ri| / |r1-r2|, where ri is point of intersection between line and ellipse
def getellipseintersect_linefrac(r1,r2,halfspan,com,dimval):
    # Translate origin to ellipse com
    r1 = r1 - com
    r2 = r2 - com
    a=halfspan[0];
    b=halfspan[1];
    if(dimval==2): #2D case
        deltavec = r2-r1
        deltasq = [number ** 2 for number in deltavec]
        A = np.dot([1/(a*a),1/(b*b)],deltasq)
        B = np.dot([2*r1[0]/(a*a),2*r1[1]/(b*b)],deltavec)
        C = r1[0]*r1[0]/(a*a) + r1[1]*r1[1]/(b*b) - 1
        determinant = B*B - 4*A*C
        if(determinant<0):
            print("imaginary solutions even in second method.",flush=True)
            print(r1);
            print(r2);
            print("Assuming both points are outside the ellipse",flush=True)
            frac1 = -100;
            frac2 = -100;
        else:
            sqrtB2m4AC = math.sqrt(determinant)
            frac1 = (-B + sqrtB2m4AC)/(2*A)
            frac2 = (-B - sqrtB2m4AC)/(2*A)
    else: #3D case
        c=halfspan[2];
        # Find intersection between 3D line and ellipsoid. Points along the line ae represented as rp(s) = r1 + s(r2-r1)
        deltavec = r2-r1
        deltasq = [number ** 2 for number in deltavec]
        A = np.dot([1/(a*a),1/(b*b),1/(c*c)],deltasq)
        B = np.dot([2*r1[0]/(a*a),2*r1[1]/(b*b),2*r1[2]/(c*c)],deltavec)
        C = r1[0]*r1[0]/(a*a) + r1[1]*r1[1]/(b*b) + r1[2]*r1[2]/(c*c) - 1
        determinant = B*B - 4*A*C
        if(determinant<0):
            print("imaginary solutions",flush=True)
            print("[A,B,C,Delta]=",flush=True)
            print([A,B,C,determinant],flush=True)
            print("r1=")
            print(r1,flush=True)
            print("r2=")
            print(r2,flush=True)
            print("[a,b,c]=")
            print(halfspan,flush=True)
            print("Assuming both points are outside the ellipse",flush=True)
            frac1 = -100;
            frac2 = -100;
        else:
            sqrtB2m4AC = math.sqrt(determinant)
            frac1 = (-B + sqrtB2m4AC)/(2*A)
            frac2 = (-B - sqrtB2m4AC)/(2*A)
    if(areEqual(frac1,0.0)):
        frac1 = 0
    elif(areEqual(frac1,1.0)):
        frac1 = 1
    if(areEqual(frac2,0.0)):
        frac2 = 0
    elif(areEqual(frac2,1.0)):
        frac2 = 1
    frac =  [frac1, frac2]
    frac.sort();
    # print(frac)
    # print([A,B,C],flush=True)
    # print(r1,flush=True)
    # print(r2,flush=True)
    return frac
    # Translate
    #x1 = x1 + xcom
    #x2 = x2 + xcom
    #y1 = y1 + ycom
    #y2 = y2 + ycom

#Written only for 2D case.
def getlinefracinsidedroplet(opt_var,sysparams):
    veccoord=getveccoord(opt_var,sysparams);
    dimval = sysparams.dimval
    a = opt_var[0];
    if(dimval==2):
        b = sysparams.Vdropfactor/a;
        halfspan=[a,b];
    else:
        b=opt_var[1];
        c=sysparams.Vdropfactor/(a*b)
        halfspan=[a,b,c];
    com=sysparams.ellipse_com;
    leninside = 0;
    lenoutside = 0;
    veccoord = getveccoord(opt_var,sysparams)
    [cyllength,vector]=getbundleprops(veccoord, sysparams)
    for idx in range(0,sysparams.Nbeads-1):
        r1=veccoord[idx,:];
        r2=veccoord[idx+1,:];
        lencyl = cyllength[idx]
        frac=getellipseintersect_linefrac(r1,r2,halfspan,com,dimval)
        f0ls0 = frac[0]<0
        f0gr1 = frac[0]>1
        f1ls0 = frac[1]<0
        f1gr1 = frac[1]>1
        f0intersect = (not f0ls0) and (not f0gr1)
        f1intersect = (not f1ls0) and (not f1gr1)
        if(f0intersect and f1intersect):
            leninside = (frac[0]+1-frac[1])*lencyl
            lenoutside = (frac[1]-frac[0])*lencyl
        elif(f0intersect or f1intersect):
            if(f0intersect):
                leninside += (1-frac[0])*lencyl
                lenoutside += frac[0]*lencyl
            else:
                leninside += frac[1]*lencyl
                lenoutside += (1-frac[1])*lencyl
        elif((f0ls0 and f1ls0) or (f0gr1 and f1gr1)):
            lenoutside += lencyl
        else:
            leninside += lencyl
    return leninside/(leninside+lenoutside)

# Energy functions

In [1]:
# Surface free energy of the droplet (VASP-PEG interface)
def getsurfaceF(opt_var,sysparams, printStatus=False):
    # Use ramanujam's approximation of perimeter of an ellipse
    # https://www.johndcook.com/blog/2013/05/05/ramanujan-circumference-ellipse/
    # https://www.mathsisfun.com/geometry/ellipse-perimeter.html
    if(sysparams.dimval==2): # in 2D, this function returns the perimeter
        a = opt_var[0];
        b = sysparams.Vdropfactor/opt_var[0];
        #lambdaval = (a-b)/(a+b)
        #pmeter= math.pi*(a+b)*(1+3*lambdaval*lambdaval/(10+math.sqrt(4-3*lambdaval*lambdaval)))
        h = (a-b)*(a-b)/((a+b)*(a+b))
        pmeter = np.pi*(a+b)*(1+h/4 + h*h/64 + h*h*h/256 + 25*h**4/16384 + 49*h**5/65536 + 441*h**6/1048576)
        if(printStatus):
            print("Surface Energy="+str(sysparams.surface_T[0]*pmeter))
        return sysparams.surface_T[0]*pmeter
    else:
        # Get ellipsoid span
        ellipsevec = [0,0,0]
        ellipsevec[0:2] = opt_var[0:2];
        ellipsevec[2] = sysparams.Vdropfactor/(ellipsevec[0]*ellipsevec[1]);
        ellipsevec.sort()
        c = ellipsevec[0];
        b = ellipsevec[1];
        a = ellipsevec[2];
        if(a==0 or b ==0 or c ==0 or sysparams.surface_T[0]==0):
            return 0
        elif(areEqual(a,b) and areEqual(b,c)):
            ravg = (a+b+c)/3
            return 4*math.pi*ravg*ravg
        else:
            csq = c*c
            asq = a*a
            bsq = b*b
            phi = math.acos(c/a)
            cossqphi = min(csq/asq,1)
            sinsqphi = 1-cossqphi
            sinphi = math.sqrt(sinsqphi)
            k = (a/b)*math.sqrt((bsq-csq)/(asq-csq))
            surfarea = 2*math.pi*csq + (2*math.pi*a*b/sinphi)*(sinsqphi*scipy.special.ellipeinc(phi, k) + cossqphi*scipy.special.ellipkinc(phi, k))
            if(printStatus):
                print("Surface Energy="+str(sysparams.surface_T[0]*surfarea))
            return sysparams.surface_T[0]*surfarea

# Stretching, Bending, and Wetting energies
def getfilamentF(opt_var,sysparams,*args):
    if(len(args)):
        printStatus = args[0]
    else:
        printStatus = False
    veccoord = getveccoord(opt_var,sysparams)
    [cyllen,vector] = getbundleprops(veccoord, sysparams)
    F = 0;
    F = getfilamentmechF(sysparams, vector, cyllen, printStatus)
    if(printStatus):
        print("Filament Mechanical energy="+str(F))
        Fmech=F;
    ellipsecoord = opt_var[0:sysparams.dimval-1];
    F += getwettingF2(ellipsecoord,veccoord, sysparams, cyllen)
    if(printStatus):
        print("Filament Wetting energy="+str(F-Fmech))
    return F

# Stretching and Bending energies
def getfilamentmechF(sysparams, vector, cyllen, *args):
    if(len(args)):
        printStatus = args[0]
    else:
        printStatus = False
    N = sysparams.Nbeads
    kbend=sysparams.kbend
    kstr = sysparams.kstr
    lcyl = sysparams.lcyl
    # stretching
    Fstr = 0;
    for idx in range(0,N-1):
        delta = cyllen[idx]-lcyl;
        Fstr += kstr*(delta*delta)
    # bending
    Fbend = 0;
    for idx in range(0,N-2):
        l1 = cyllen[idx]
        l2 = cyllen[idx+1]
        if(areEqual(l1*l2,0.0)):
            x = 1
        else:
            # Calculate dot product
            x = np.dot(vector[idx,:],vector[idx+1,:])/(l1*l2)
        if x<-1.0:
            x=-1.0;
        if(x>1.0):
            x = 1.0;
        Fbend += kbend*(1-x)
    # Calculate stretching + bending energy
    if(printStatus):
        print("Filament Stretching energy="+str(Fstr))
        print("Filament Bending energy="+str(Fbend))
    return Fstr+Fbend

#To be used only to check failure.
def getfilamentstrF(sysparams, vector, cyllen):
    N = sysparams.Nbeads
    kstr = sysparams.kstr
    lcyl = sysparams.lcyl
    # stretching
    Fstr = 0;
    for idx in range(0,N-1):
        delta = cyllen[idx]-lcyl;
        Fstr += kstr*(delta*delta)
    return Fstr

#Wetting energies
def getwettingF2(ellipsecoord, coord, sysparams, cyllen, printStatus = False):
    dimval = sysparams.dimval;
    gamma_fil_drop = sysparams.surface_T[1];
    gamma_fil_dilute = sysparams.surface_T[2];
    if(gamma_fil_drop == 0 and gamma_fil_dilute == 0):
        return 0;
    wettingenergy = 0
    rcyl = sysparams.rcyl
    a = ellipsecoord[0]
    if(dimval==3):
        b = ellipsecoord[1]
        #c = ellipsecoord[2]
        c = sysparams.Vdropfactor/(a*b)
        Wenergyfactor = 2*math.pi*rcyl
    else:
        b = sysparams.Vdropfactor/ellipsecoord[0]
        Wenergyfactor = rcyl;
    com = sysparams.ellipse_com
    eavgvec = []
    N = sysparams.Nbeads
    for idx in range(0,N-1):
        #print([N,idx,3*(idx+1),3*(idx+2),len(coord)])
        r1 = coord[idx,:]
        r2 = coord[idx+1,:]
        # Translate origin to ellipse com
        r1 = r1 - com
        r2 = r2 - com
        # Interpolate
        delta = r2 - r1
        # Accomodate 4 points per cylinder - can be tuned later
        discretize_len = sysparams.lcyl/4
        lcylcurr = cyllen[idx]
        if(lcylcurr ==0):
            #
            #print(cyllen)
            print('FAIL!')
            #print(lcylcurr)
            continue
        ds = discretize_len/lcylcurr
        s = 0
        sprev = 0
        evec = []
        dlvec = []
        nsegments = int(lcylcurr/discretize_len)
        deltasq = [number ** 2 for number in delta]
        # Eccentricity of points along a line joining r1 and r2
        #Expression is obtained by rewriting the ellipse coordinates in terms of r(s) = r1+s(r2-r1)
        if(dimval==3):
            A = np.dot([1/(a*a),1/(b*b),1/(c*c)],deltasq)
            B = np.dot([2*r1[0]/(a*a),2*r1[1]/(b*b),2*r1[2]/(c*c)],delta)
            C = r1[0]*r1[0]/(a*a) + r1[1]*r1[1]/(b*b) + r1[2]*r1[2]/(c*c)
        else:
            A = np.dot([1/(a*a),1/(b*b)],deltasq)
            B = np.dot([2*r1[0]/(a*a),2*r1[1]/(b*b)],delta)
            C = r1[0]*r1[0]/(a*a) + r1[1]*r1[1]/(b*b)
        if(printStatus):
            coord_temp=[];
            svec=[]
            r1vec=[]
            r2vec=[]
        for nsg in range(0,nsegments+1):
            if(printStatus):
                r = r1+s*delta
                coord_temp.append(r)
                svec.append(s)
                r1vec.append(r1)
                r2vec.append(r2)
            if(s>0):
                dl = (s-sprev)*lcylcurr
                dlvec.append(dl)
            #eccentricity is x^2/a^2 + y^2/b^2 + z^2/c^2
            evec.append(s*s*A+s*B+C)
            sprev = s
            s = s + ds
        # if the for loop did not cover the entire rod, we need to account for the rest
        if(~areEqual(sprev,1.0)):
            s = 1
            if(printStatus):
                svec.append(s)
                coord_temp.append(r1+delta)
            #r = r2
            dl = (s-sprev)*lcylcurr
            dlvec.append(dl)
            #eccentricity is x^2/a^2 + y^2/b^2 + z^2/c^2
            evec.append(A+B+C)
        evec=np.asarray(evec)
        eavg = 0.5*(evec[0:len(evec)-1:1]+evec[1:len(evec):1])
        #eavgvec[len(eavgvec):len(eavgvec)+len(eavg)] = eavg
        dlvec=np.asarray(dlvec)
        gamma = getsurfaceTension(eavg, sysparams)
        wettingenergy = wettingenergy + Wenergyfactor*np.sum(np.multiply(dlvec,gamma))
        if(printStatus):
            print([idx,N])
            print(evec)
            print(eavg)
            print(gamma)
            print([A,B,C])
            print([a,b])
            print(r1vec)
            print(r2vec)
            print(svec)
            #print(coord_temp)
    return wettingenergy

def gettotalenergy(opt_var,sysparams, printStatus=False):
    return getfilamentF(opt_var,sysparams, printStatus) + getsurfaceF(opt_var,sysparams, printStatus)

# Initialize and pertrub bundle coordinates

In [ ]:
    ######## Initialization functions ########
def initializezigzag(sysparams):
    dimval = sysparams.dimval
    opt_var = np.zeros(dimval+dimval*sparams.Nbeads)
    theta = np.radians(45)
    lcyl = sysparams.lcyl;
    xval = 200+np.random.random_sample()/10;
    for idx in range(0,sparams.Nbeads):
        xval = xval + lcyl*np.cos(theta)+np.random.random_sample()
        # if even
        yval = 200+np.random.random_sample()/10;
        if(idx%2==1):
            yval = yval+lcyl*np.sin(theta)+np.random.random_sample()
        opt_var[dimval+dimval*idx] = xval;
        opt_var[dimval+dimval*idx+1] = yval;
        opt_var[dimval+dimval*idx+2] = yval;
    return opt_var

def initializestraight(sysparams, r1, Lbundle):
    r1 = np.asarray(r1)
    dimval = sysparams.dimval
    opt_var = np.zeros(dimval+dimval*sparams.Nbeads)
    Nsegments = sysparams.Nbeads - 1
    lcyl = sysparams.lcyl;
    dir = np.asarray([np.random.random_sample(),np.random.random_sample(),np.random.random_sample()])
    dir = dir/np.linalg.norm(dir)
    r2 = Lbundle*dir + r1
    ds = sysparams.lcyl/(Lbundle)
    opt_var[range(3,6,1)] = r1
    s = ds;
    delta = r2-r1
    for idx in range(0,Nsegments):
        opt_var[dimval*(idx+2):dimval*(idx+3)] = r1 + s*delta
        s = s + ds
    opt_var = addrandomnoise(opt_var,sysparams.noisefactor)
    return opt_var

def initializestraight2(sysparams, r1, rend, Lbundle):
    slideval = sysparams.dimval - 1;
    r1 = np.asarray(r1)
    rend = np.asarray(rend)
    dimval = sysparams.dimval
    opt_var = np.zeros(slideval+dimval*sysparams.Nbeads)
    Nsegments = sysparams.Nbeads - 1
    lcyl = sysparams.lcyl;
    dir = rend-r1
    dir = dir/np.linalg.norm(dir)
    r2 = Lbundle*dir + r1
    ds = sysparams.lcyl/(Lbundle)
    # in 3D case, you only need a and b. c can be interpretted from constant volume constraint
    # So, coordinates of bundle starts at dimval - 1.
    opt_var[range(dimval-1,2*dimval-1,1)] = r1
    s = ds;
    delta = r2-r1
    for idx in range(0,Nsegments):
        opt_var[slideval+dimval*(idx+1):slideval+dimval*(idx+2)] = r1 + s*delta
        s = s + ds
    opt_var = addrandomnoise(opt_var,sysparams.noisefactor)
    return opt_var

def extendstraight(sysparams, opt_var):
    Nbeads = sysparams.Nbeads
    dimval = sysparams.dimval
    slideval = dimval - 1;
    veccoord=getveccoord(opt_var, sysparams)
    r1 = veccoord[Nbeads-2,:]
    r2 = veccoord[Nbeads-1,:]
    opt_vartemp = np.zeros(slideval + (Nbeads+1)*dimval)
    opt_vartemp[0:slideval+(Nbeads)*dimval] = opt_var
    dir = r2-r1
    dir = dir/np.linalg.norm(dir)
    rnew = sysparams.lcyl*dir + r2;
    opt_vartemp[slideval+(Nbeads)*dimval:slideval+(Nbeads+1)*dimval] = rnew
    opt_vartemp = addrandomnoise(opt_vartemp,sysparams.noisefactor)
    return opt_vartemp

def addrandomnoise(vec,noisefactor):
    rnd = np.random.random_sample(len(vec))/noisefactor;
    return vec+rnd;

# Simulate

In [ ]:
# Sample input file. Edit this to generate results over multiple conditions.
diavec=np.asarray([1.5])
Nfil=np.asarray([200])
gamma_VP=10.0 #Surface tension of VASP and surrounding PEG medium
main.simulate(2,diavec,Nfil,gamma_VP,0,True,1e-06, False,10000.0)

## Generate a ring

In [ ]:
#This code tries to produce a ring with 1 filament and then slowly increases its bending stiffness
def generatering(sparams):
    aspect_ratio_cndn=False
    while not(aspect_ratio_cndn):
        for liter in range(0,len(lcylvec)):
            sparams.lcyl = lcylvec[liter]
            print("Lcyl="+str(sparams.lcyl), flush=True)
            for kiter in range(0,len(Nfilvec)):
                #sparams.rcyl = R_fil*Nfil[kiter]
                sparams.kbendfil = (Lp_fil*4.1/sparams.lcyl)
                sparams.kbend = Nfilvec[kiter]*sparams.kbendfil
                print("kbend="+str(sparams.kbend), flush=True)
                for fiter in range(0,len(fTolvec)):
                    ftolval = fTolvec[fiter]
                    print("fTol="+str(ftolval), flush=True)
                    for diter in range(0,len(diavec)):
                        diameter = diavec[diter]
                        radiusnm = diameter*1e3/2
                        if(sparams.dimval==2):
                            sparams.ellipse=[radiusnm,radiusnm]
                            sparams.Vdropfactor = sparams.ellipse[0]*sparams.ellipse[1]
                        else:
                            sparams.ellipse=[radiusnm,radiusnm,radiusnm]
                            sparams.Vdropfactor = sparams.ellipse[0]*sparams.ellipse[1]*sparams.ellipse[2]
                        Nreps=int((np.pi-1)*diameter/(sparams.lcyl*1e-3))
                        print("Nreps="+str(Nreps),flush=True)
                        initsurfarea=np.zeros(Nreps);
                        finlsurfarea=np.zeros(Nreps);
                        initvol=np.zeros(Nreps);
                        finlvol=np.zeros(Nreps);
                        print("Diameter="+str(diameter),flush=True)
                        completed_status = False
                        while(completed_status==False):
                            solnlistvec=[];
                            sparams.Nbeads = int(diameter*1e3/sparams.lcyl)
                            print("Nbeads="+str(sparams.Nbeads),flush=True)
                            Ncyl = sparams.Nbeads -1
                            for trialidx in range(0,Nreps):
                                print(str(sparams.kbend)+', '+str(trialidx),flush=True)
                                status = False
                                completed_status = False
                                if(trialidx==0):
                                    if(sparams.dimval==2):
                                        rinit = np.array(sparams.ellipse_com) - np.array([radiusnm, 0])
                                        rfinal = np.array(sparams.ellipse_com) + np.array([radiusnm, 0])
                                    else:
                                        rinit = np.array(sparams.ellipse_com) - np.array([radiusnm, 0, 0])
                                        rfinal = np.array(sparams.ellipse_com) + np.array([radiusnm, 0, 0])
                                    print(rinit,flush=True)
                                    print(rfinal,flush=True)
                                    init_config = initializestraight2(sparams,rinit, rfinal,sparams.lcyl*(sparams.Nbeads-1))
                                    opt_var = init_config;
                                    opt_var[0:dimval-1] = sparams.ellipse[0:dimval-1]
                                    print(opt_var,flush=True)
                                else:
                                    opt_var = extendstraight(sparams, opt_var)
                                    sparams.Nbeads += 1
                                #print(opt_var,flush=True)
                                #solnlistvec.append(opt_var)
                                while(status==False):
                                    print("Initial Energy="+str(gettotalenergy(opt_var,sparams,True)))
                                    # Minimize
                                    while(status==False):
                                        #res = minimize(gettotalenergy, opt_var, args=sparams, method='SLSQP', options={'ftol': 1e-4, 'maxiter': 1e4, 'eps':1e-4,'disp': False})
                                        res = minimize(gettotalenergy, opt_var, args=sparams, method='SLSQP',bounds=bnds, options={'ftol': ftolval, 'maxiter': 1e4, 'eps':ftolval,'disp': False})
                                        status = res.success
                                        print("Success= "+str(res.success))
                                        opt_var = res.x
                                        finalE = gettotalenergy(res.x,sparams,True)
                                        print("Final Energy="+str(finalE))
                                        if(not(res.success)):
                                            print(res.message)
                                            failcounter = failcounter + 1;
                                        if(failcounter>=failtol):
                                            #if stretching accounts for more than 10% of total energy
                                            veccoord = getveccoord(res.x,sparams);
                                            [cyllength,vector,cylaxis,com]=getbundleprops(veccoord, sparams, True, True)
                                            Fstr = getfilamentstrF(sparams, vector, cyllength);
                                            wettingfraction = getlinefracinsidedroplet(res.x,sparams)
                                            print("Stretching Energy="+str(Fstr),flush=True)
                                            print("Total Energy="+str(finalE),flush=True)
                                            print("Wetting fraction="+str(wettingfraction),flush=True)
                                            if(Fstr>finalE*failEstrtol or wettingfraction<=0.95):
                                                print("Fail limit reached. Exiting while loop.",flush=True)
                                                break;
                                            else:
                                                print("Cylinder lengths are fine and are inside the droplet. Accepting current solution.",flush=True)
                                                failcounter = 0;
                                                status=True;
                                    if(failcounter>=failtol):
                                        print("Fail limit reached. Exiting while loop.",flush=True)
                                        break;
                                wettingfraction = getlinefracinsidedroplet(res.x,sparams);
                                print("wettingfraction="+str(wettingfraction),flush=True)
                                cndn = not(areEqual(wettingfraction,1.0))
                                if(failcounter>=failtol or cndn):
                                    if(cndn):
                                        print("Cylinder is exposed to dilute medium. Wetting fraction="+str(wettingfraction)+". Resetting trialidx",flush=True)
                                        veccoord = getveccoord(res.x,sparams);
                                        [cyllength,vector,cylaxis,com]=getbundleprops(veccoord, sparams, True, True)
                                        temp=res.x;
                                        x=getwettingF2(temp[0:2], veccoord, sparams, cyllength)
                                        print(res.x)
                                    else:
                                        print("Fail limit reached. Resetting trialidx.",flush=True)
                                    solnlistvec=[];
                                    failcounter = 0;
                                    trialidx = 0;
                                    break;
                                counter = counter + 2;
                                if(sparams.dimval==3):
                                    print(["initial volume="+str(getellipsevolume3D(opt_var))])
                                    print(["Final volume="+str(getellipsevolume3D(res.x))])
                                else:
                                    print(["initial volume="+str(getellipsevolume(opt_var))])
                                    print(["Final volume="+str(getellipsevolume(res.x))])
                                #print(res.x,flush=True)
                                #print(res.nit)
                                if(res.success):
                                    #solnlistvec.append(res.x)
                                    completed_status = True;
                        if(failcounter>=failtol):
                            print("Fail limit reached. Resetting Diameter.",flush=True)
                            break;
                        print("Printing cylinder lengths",flush=True)
                        veccoord = getveccoord(res.x,sparams);
                        [cyllength,vector,cylaxis,com]=getbundleprops(veccoord, sparams, True, True)
                        print(cyllength)
                    if(failcounter>=failtol):
                        print("Fail limit reached. Resetting fTol.",flush=True)
                        failcounter = 0;
                        break;
                if(failcounter>=failtol):
                    print("Fail limit reached. Resetting kiter.",flush=True)
                    break;
            if(failcounter>=failtol):
                print("Fail limit reached. Resetting lcyl.",flush=True)
                break;
        soln = res.x
        a=soln[0];
        b=sparams.Vdropfactor/soln[0]
        aratio=max(a,b)/min(a,b)
        if(aratio<=1.1):
            aspect_ratio_cndn=True;
            print("Aspect ratio of ring in droplet is "+str(aratio)+". Success.")
        else:
            print("Aspect ratio of ring in droplet is "+str(aratio)+". Trying to create ring again.")
    return res

## Increase bundle thickness

In [ ]:
#This code tries to produce a ring with 1 filament and then slowly increases its bending stiffness
def simulate(dimval, diavec,targetNfil,gamma_VP,trial, *args):
    # args - lcylscalewithD, fTol, nonlinear, alpha
    if(len(args)):
        lcylscalewithD = args[0]
    else:
        lcylscalewithD = False
    if(len(args)>=2):
        fTolvec = [args[1]]
    else:
        fTolvec = [1e-8]
    if(len(args)>=3):
        nonlinearbool=args[2];
    if(len(args)>=4):
            alphaval=args[3];
    if(len(diavec)>1):
        print("Error! diameter values cannot be multielement arrays. Exiting.")
        sys.exit(1);
    sparams = SystemParams();
    sparams.noisefactor = 1;
    sparams.dimval = dimval
    sparams.nonlinearbool=nonlinearbool;
    sparams.alpha=alphaval;
    bnds = Bounds(1e-9,1e10);
    # VASP-PEG, VASP-Actin, PEG-Actin
    #sparams.surface_T = [1e1,1e-8,0.5e4]
    sparams.surface_T = [gamma_VP,1e-8,0.5e4]
    #sparams.surface_T = [1e1,1e-8,0.5e3]
    sparams.rcyl = 5
    dimval = sparams.dimval
    plotStatus = False
    counter = 1;
    if(sparams.dimval==2):
        sparams.ellipse_com=[8e3,8e3]
    else:
        sparams.ellipse_com=[8e3,8e3,8e3]
    Lp_fil = 17.73*1e3;#persistence length in nm
    R_fil = 15;
    Nfilvec = np.array([1])
    #diavec=[1.0, 2.0, 3.0, 4.0]# in microns
    lcylvec = [100]
    sparams.kstr = 1e2;
    if(lcylscalewithD):
        #Assume bundle length increases by 10% of diameter during each iteration
        lcylvec = [diavec[0]*0.1*1e3]#convert from microns to nanometer
    failcounter = 0;
    failtol = 3;
    failEstrtol = 0.1;
    if(sparams.nonlinearbool==False):
        filetag='_KD'
    else:
        filetag='NL_KD'
    Atagstring='_A_'+str(sparams.alpha)
    sys.stdout = open('outfileV2'+filetag+str(diavec[0])+'_N'+str(targetNfil[0])+'_G_'+str(gamma_VP)+Atagstring+'fTol'+str(fTolvec[0])+'T'+str(trial)+'.txt', 'w')
    sys.sterr = open('errfileV2'+filetag+str(diavec[0])+'_N'+str(targetNfil[0])+'_G_'+str(gamma_VP)+Atagstring+'fTol'+str(fTolvec[0])+'T'+str(trial)+'.txt', 'w')
    print("fTol="+str(fTolvec[0]),flush=True)
    print("alpha="+str(alphaval),flush=True)
    #Start by creating a ring with 1 filament
    print("Creating a ring with one filament", flush=True)
    for reps in range(trial,trial+1):
        print("-------------------------")
        print("Replicate="+str(reps), flush=True)
        opt_var=generatering(sparams);

        print("Increasing number of filaments.", flush=True)
        print("Initial coord size "+str(len(opt_var)))
        print(opt_var,flush=True)
        for Nfil in range(2,targetNfil[0]+1):
            sparams.Nfil = Nfil;
            # add random noise
            print("Nfil="+str(Nfil),flush=True)
            opt_var = addrandomnoise(opt_var,sparams.noisefactor);
            print("------------",flush=True)
            sparams.rcyl = R_fil*(Nfil)
            #This step is handled automatically when calculating bending energy.
            sparams.kbend = getkbend(sparams);
            status=False;
            failcounter=0;
            solnlistvec.append(opt_var)
            print("Initial Energy="+str(gettotalenergy(opt_var,sparams,True)))
            # Minimize
            while(status==False):
                res = minimize(gettotalenergy, opt_var, args=sparams, method='SLSQP',bounds=bnds, options={'ftol': ftolval, 'maxiter': 1e4, 'eps':ftolval,'disp': False})
                status = res.success
                print("Success= "+str(res.success))
                opt_var = res.x
                finalE = gettotalenergy(res.x,sparams,True)
                print("Final Energy="+str(finalE))
                if(not(res.success)):
                    print(res.message)
                    failcounter = failcounter + 1;
                else:
                    solnlistvec.append(res.x)
                    print(res.x,flush=True)
                if(failcounter>=failtol):
                    #if stretching accounts for more than 10% of total energy
                    veccoord = getveccoord(res.x,sparams);
                    [cyllength,vector,cylaxis,com]=getbundleprops(veccoord, sparams, True, True)
                    Fstr = getfilamentstrF(sparams, vector, cyllength);
                    wettingfraction = getlinefracinsidedroplet(res.x,sparams)
                    print("Stretching Energy="+str(Fstr),flush=True)
                    print("Total Energy="+str(finalE),flush=True)
                    print("Wetting fraction="+str(wettingfraction),flush=True)
                    if(Fstr>finalE*failEstrtol or wettingfraction<=0.95):
                        print("Fail limit reached. Exiting while loop.",flush=True)
                        break;
                    else:
                        print("Cylinder lengths are fine and are inside the droplet. Accepting current solution.",flush=True)
                        failcounter = 0;
                        status=True;
                    wettingfraction = getlinefracinsidedroplet(res.x,sparams);
                    print("wettingfraction="+str(wettingfraction),flush=True)
                    cndn = not(areEqual(wettingfraction,1.0))
                    if(failcounter>=failtol or cndn):
                        if(cndn):
                            print("Cylinder is exposed to dilute medium. Wetting fraction="+str(wettingfraction)+". Try generating a new ring",flush=True)
                            veccoord = getveccoord(res.x,sparams);
                            [cyllength,vector,cylaxis,com]=getbundleprops(veccoord, sparams, True, True)
                            temp=res.x;
                            x=getwettingF2(temp[0:2], veccoord, sparams, cyllength)
                            print(res.x)
                        else:
                            print("Fail limit reached. Try generating a new ring.",flush=True)
                        solnlistvec=[];
                        failcounter = 0;
                        trialidx = 0;
                        break;
            if(res.success):
                completed_status = True;
                print("Printing cylinder lengths",flush=True)
                veccoord = getveccoord(res.x,sparams);
                [cyllength,vector,cylaxis,com]=getbundleprops(veccoord, sparams, True, True)
                print(cyllength)

        if(Nfil==targetNfil):
            textfile = open("solution_TV2_"+str(reps)+"_D_"+str(diameter)+"_N_"+str(Nfil)+"_lcyl_"+str(round(lcylvec[liter],2))+"_G_"+str(gamma_VP)+Atagstring+"_fTol_"+str(ftolval)+'.txt', "w")
            for element in solnlistvec:
                stringvec=""
                for values in element:
                    stringvec+=str(values)+","
                textfile.write(stringvec + "\n")
            textfile.close()
    sys.stdout.close()
    sys.stderr.close()